## В этом домашнем задании вы сделаете первые шаги в мире линейной бинарной классификации!

In [2]:
import numpy as np
import pandas as pd

#### Задание 1

Мы будем работать с данными **Microsoft Malware Detection**

Таргетом будет последний столбец `HasDetection`, который принимает значения $\{0,\, 1\}$ в случае отсутствия или наличия вируса на компьютере соответственно. Признаками будут выступать всевозможные характеристики компьютера.

In [142]:
data = pd.read_csv('Data/train.csv')

C:\Users\merku\AppData\Local\Temp\ipykernel_21068\3943152542.py:1: DtypeWarning: Columns (28) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv('Data/train.csv')


In [121]:
data.head()

,IsBeta,RtpStateBitfield,IsSxsPassiveMode,DefaultBrowsersIdentifier,AVProductStatesIdentifier,AVProductsInstalled,AVProductsEnabled,HasTpm,CountryIdentifier,CityIdentifier,OrganizationIdentifier,GeoNameIdentifier,LocaleEnglishNameIdentifier,OsBuild,OsSuite,IsProtected,AutoSampleOptIn,SMode,IeVerIdentifier,Firewall,UacLuaenable,Census_OEMNameIdentifier,Census_OEMModelIdentifier,Census_ProcessorCoreCount,Census_ProcessorManufacturerIdentifier,Census_ProcessorModelIdentifier,Census_PrimaryDiskTotalCapacity,Census_SystemVolumeTotalCapacity,Census_HasOpticalDiskDrive,Census_TotalPhysicalRAM,Census_InternalPrimaryDiagonalDisplaySizeInInches,Census_InternalPrimaryDisplayResolutionHorizontal,Census_InternalPrimaryDisplayResolutionVertical,Census_InternalBatteryNumberOfCharges,Census_OSBuildNumber,Census_OSBuildRevision,Census_OSInstallLanguageIdentifier,Census_OSUILocaleIdentifier,Census_IsPortableOperatingSystem,Census_IsFlightingInternal,Census_IsFlightsDisabled,Census_ThresholdOptIn,Census_FirmwareManufacturerIdentifier,Census_FirmwareVersionIdentifier,Census_IsSecureBootEnabled,Census_IsWIMBootEnabled,Census_IsVirtualDevice,Census_IsTouchEnabled,Census_IsPenCapable,Census_IsAlwaysOnAlwaysConnectedCapable,Wdft_IsGamer,Wdft_RegionIdentifier,HasDetections
0,0,7.0,0,1632.0,53447.0,1.0,1.0,1,29,128035.0,18.0,35.0,171,17134,256,1.0,0,0.0,137.0,1.0,1.0,2668.0,9124.0,4.0,5.0,2341.0,476940.0,299451.0,0,4096.0,18.9,1440.0,900.0,4.294967e+09,17134,165,26.0,119,0,0.0,0.0,0.0,628.0,36144.0,0,0.0,0.0,0,0,0.0,0.0,10.0,0
1,0,7.0,0,1632.0,53447.0,1.0,1.0,1,93,1482.0,18.0,119.0,64,17134,256,1.0,0,0.0,137.0,1.0,1.0,2668.0,91656.0,4.0,5.0,2405.0,476940.0,102385.0,0,4096.0,13.9,1366.0,768.0,1.000000e+00,17134,1,8.0,31,0,0.0,0.0,0.0,628.0,57858.0,0,0.0,0.0,0,0,0.0,0.0,8.0,0
2,0,7.0,0,1632.0,53447.0,1.0,1.0,1,86,153579.0,18.0,64.0,49,17134,768,1.0,0,0.0,137.0,1.0,1.0,4909.0,317701.0,4.0,5.0,1972.0,114473.0,113907.0,0,4096.0,21.5,1920.0,1080.0,4.294967e+09,17134,165,7.0,30,0,0.0,0.0,0.0,142.0,52682.0,0,0.0,0.0,0,0,0.0,0.0,3.0,0
3,0,7.0,0,1632.0,53447.0,1.0,1.0,1,88,20710.0,27.0,117.0,115,17134,256,1.0,0,0.0,137.0,1.0,1.0,1443.0,275890.0,4.0,5.0,2273.0,238475.0,227116.0,0,4096.0,18.5,1366.0,768.0,4.294967e+09,17134,228,17.0,64,0,0.0,0.0,0.0,355.0,20050.0,0,0.0,0.0,0,0,0.0,0.0,3.0,1
4,0,7.0,0,1632.0,53447.0,1.0,1.0,1,18,37376.0,27.0,277.0,75,17134,768,1.0,0,0.0,137.0,1.0,1.0,1443.0,331929.0,4.0,5.0,2500.0,476940.0,101900.0,0,6144.0,14.0,1366.0,768.0,0.000000e+00,17134,191,8.0,31,0,0.0,0.0,0.0,355.0,19844.0,0,0.0,0.0,0,0,0.0,0.0,1.0,1


Удалите константные признаки и признаки `ProductName` `MachineIdentifier`

In [99]:
data = pd.read_csv('Data/train.csv')

C:\Users\merku\AppData\Local\Temp\ipykernel_21208\3943152542.py:1: DtypeWarning: Columns (28) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv('Data/train.csv')


In [100]:
### Your code is here
data.drop(columns=['ProductName', 'MachineIdentifier'], inplace=True)

check = data.nunique() == 1
data.drop(columns=data.loc[:, check.values].columns, inplace=True)

Посмотрите на соотношение классов в таргете. Все ли хорошо?

In [101]:
print(sum(data['HasDetections'] == 1), '- positive class,')
print(sum(data['HasDetections'] == 0), '- negative class')

100060 - positive class,
99940 - negative class


Ответьте на вопрос: почему с вашей точки зрения важно иметь представление о балансе классов в ваших данных?

Избавьтесь от пропусков в данных! 

Новый для нас прием: признаки с более чем половиной пропусков следует удалить.

Согласитесь, если в вашей колонке среди 100 объектов всего лишь у 2 есть какое-то непропущенное значение, странно все остальные заполнять средним от этих двух чисел. Такие "редкие" признаки лучше вообще опустить!


В категориальных колонках заменим отсутствующую категорию просто некоторой новой и назовем ее `NaN`.

А в числовых, ради разнообразия, заполним пропуски медианным значением.

Создайте копию полученного датафрейма и положите ее в переменную data_2. Понадобится в следующих заданиях.

In [102]:
### Your code is here
data.dropna(thresh=data.shape[0] / 2, axis=1, inplace=True)

obj_cols = data.select_dtypes('object').columns
data[obj_cols] = data[obj_cols].fillna('NaN')

num_cols = data.select_dtypes('number').columns
grouped_med = data.groupby('HasDetections')[num_cols].transform("median")
data[num_cols] = data[num_cols].fillna(grouped_med)

In [103]:
data_2 = data.copy()

In [91]:
data.shape[1]

73

Так же поработаем над всеми категориальными колонкам перед запуском непосредственно моделей.

Провернем самый базовый и наглый метод - несмотря на количество уникальных значений в каждой категории, просто применим ко всей категориальной части датасета `OneHotEncoding`

In [92]:
data = pd.get_dummies(data, columns=data.select_dtypes('object').columns, drop_first=True)
data.shape

(200000, 6114)

In [48]:
# oh_data = pd.get_dummies(data_1, columns=data_1.select_dtypes('object').columns, drop_first=True)
# 
# data_1 = pd.concat([data_1.select_dtypes('number'), oh_data], axis=1)
# data_1 = data_1.loc[:, ~data_1.columns.duplicated()]
# data_1.shape

###  Разделим выборку на тренировочную и тестовую

P.S. в задачах классификации, как и в задаче регрессии, можно использовать технологию Кросс-Валидации. Например, по одному из двух следующих сценариев:

1) Отделить валидацию и тест, произвести подбор лучшей модели с помощью `K-Fold` на валидации, финально обучить выбранную модель на всей валидации и замерить качество на заранее отложенном финальном тесте!

2) Всю выборку назвать валидационной и на ней применить `K-Fold` без финального замера.

В этой домашней работе попросим Вас быть еще проще! :)
Реализуем просто технологию отложенной выборки в пропорции 3:1

In [93]:
from sklearn.model_selection import train_test_split

X = data.drop(columns=['HasDetections'])
y = data['HasDetections']

X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                    test_size=0.25,
                                                    shuffle=True,
                                                    random_state=1)

Соберите `Pipeline`, реализовав в нем 2 шага: стандартизация данных через `StandardScaler` и обучение логистической регрессии с помощью `LogisticRegression`, положите результаты в переменную `pipe`, а в классе модели `LogisticRegression` укажите параметр `penalty='none'`

In [94]:
### Your code is here

from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler

pipe = Pipeline(steps=[('scaler', StandardScaler()),
                       ('LR', LogisticRegression(penalty=None))])

Чтобы замерить качество работы такой модели на трейне и на тесте воспользуемся функцией `cross_validate`

Вопрос: что передавать ей в параметр cv? Ведь мы уже разделили нашу выборку на трейн и тест (имеем всего 1 fold). Для этого можно просто передать список от кортежа, содержащего индексы тренировочных и тестовых объектов.

In [95]:
from sklearn.model_selection import cross_validate
import datetime

custom_cv = [(X_train.index.to_list(), X_test.index.to_list())]

begin_time = datetime.datetime.now()

cv_result_pipe = cross_validate(pipe, X, y, scoring='accuracy',
                                cv=custom_cv, return_train_score=True)

print(f"Accuracy на трейне: {np.mean(cv_result_pipe['train_score']).round(3)}")
print(f"Accuracy на тесте: {np.mean(cv_result_pipe['test_score']).round(3)}")

print(f"Время работы алгоритма: {datetime.datetime.now() - begin_time}")

KeyboardInterrupt: 

Что можете сказать про время работы алгоритма?

Очевидно, оно достаточно большие. Уж тем более для линейных моделей.

Такое как раз-таки происходит из-за того, что количество фичей, который мы передали нашей модели - гигантское! Классу требуется много времени и памяти, чтобы обработать датасет.

Поэтому те колонки, в которых количество уникальных категорий превышает какое-то адекватное число, следует кодировать иначе, нежели с помощью технологии `One-Hot-Encoding`.

Теперь вы верите, что более умные кодировки зачастую прям необходимы! Раньше мы этот факт не демонстрировали!

Дело еще вот в чем: в классе `LogisticRegression`, как и, например, `Lasso`, есть параметр, ограничивающий максимальное количество итераций во время обучения модели. Так, если данных много и итераций тоже ожидается большое число, найденная разделяющая гиперплоскость может оказаться не самой лучшей, так как наш алгоритм (будь то градиентный спуск или любой иной) просто 'не доползет'. 

#### Задание 2

Теперь попробуем другой метод кодирования категориальных колонок, а именно счётчики.
Построем ту же модель и на том же разделении, просто заново иначе переобработаем датасет. 

Для тех категориальных признаков, у которых количество уникальных значений в колоночках больше 5, применим `MeanTargetEncoding`.

Для остальных оставим любимый `OneHotEncoding` (как делали на практике и в предыдущем уроке).

In [104]:
### Your code is here

less_than_5_cols = data_2.select_dtypes('object').loc[:, data_2.nunique() <= 5].columns.values
more_than_5_cols = data_2.select_dtypes('object').loc[:, data_2.nunique() > 5].columns.values

data_2 = pd.get_dummies(data_2, columns=less_than_5_cols, drop_first=True)

for col in more_than_5_cols:
    mean_target = data_2.groupby(col)['HasDetections'].mean()
    data_2[col] = data_2[col].map(mean_target)

In [106]:
X_2 = data_2.drop(columns=['HasDetections'])
y_2 = data_2['HasDetections']

X_2_train, X_2_test, y_2_train, y_2_test = train_test_split(X_2, y_2,
                                                            test_size=0.25,
                                                            shuffle=True,
                                                            random_state=1)

Опять обучим модель, пока что без изменений! Скажите, стало ли быстрее? А что с качеством?

In [107]:
### Your code is here

from sklearn.model_selection import cross_validate
import datetime

custom_cv = [(X_2_train.index.to_list(), X_2_test.index.to_list())]

begin_time = datetime.datetime.now()

cv_result_pipe = cross_validate(pipe, X_2, y_2, scoring='accuracy',
                                cv=custom_cv, return_train_score=True)

print(f"Accuracy на трейне: {np.mean(cv_result_pipe['train_score']).round(3)}")
print(f"Accuracy на тесте: {np.mean(cv_result_pipe['test_score']).round(3)}")

print(f"Время работы алгоритма: {datetime.datetime.now() - begin_time}")

Accuracy на трейне: 0.638
Accuracy на тесте: 0.638
Время работы алгоритма: 0:00:02.526585


#### Задание 3: Регуляризация

Как и в моделях регрессии, решая задачу классификации, можем штрафовать минимизируемый функционал за большие веса, добавив к нему L1 или L2 норму весов (все как раньше!).

Для этого в изначальном классе `LogisticRegression` изменить параметр `penalty` на l1 или l2 соответственно. Выберите второй вариант! Можно воспользоваться методом `set_params` и применить его к `pipe`.

In [110]:
### Your code is here

# pipe.set_params(LR__penalty='l1')
pipe.set_params(LR__penalty='l2')

Pipeline(steps=[('scaler', StandardScaler()), ('LR', LogisticRegression())])

Теперь наша модель будет строить логистическую регрессию с L2 регуляризатором! Помним, что у регуляризируемых моделей есть гиперпараметр, контролирующий силу регуляризации, который выбирается ДО запуска метода fit, то есть заранее, когда модель еще не обучена. 

Конечно же, выбор этого параметра влияет итоговые результаты. Хочется поперебирать различные параметры регуляризации и найти такой, при котором качество на тесте окажется лучшим! 

Сгенерируем массив гиперпараметра, которые планируем перебрать:

In [113]:
alphas = np.linspace(0.01, 100, 100)
alphas

array([1.000e-02, 1.020e+00, 2.030e+00, 3.040e+00, 4.050e+00, 5.060e+00,
       6.070e+00, 7.080e+00, 8.090e+00, 9.100e+00, 1.011e+01, 1.112e+01,
       1.213e+01, 1.314e+01, 1.415e+01, 1.516e+01, 1.617e+01, 1.718e+01,
       1.819e+01, 1.920e+01, 2.021e+01, 2.122e+01, 2.223e+01, 2.324e+01,
       2.425e+01, 2.526e+01, 2.627e+01, 2.728e+01, 2.829e+01, 2.930e+01,
       3.031e+01, 3.132e+01, 3.233e+01, 3.334e+01, 3.435e+01, 3.536e+01,
       3.637e+01, 3.738e+01, 3.839e+01, 3.940e+01, 4.041e+01, 4.142e+01,
       4.243e+01, 4.344e+01, 4.445e+01, 4.546e+01, 4.647e+01, 4.748e+01,
       4.849e+01, 4.950e+01, 5.051e+01, 5.152e+01, 5.253e+01, 5.354e+01,
       5.455e+01, 5.556e+01, 5.657e+01, 5.758e+01, 5.859e+01, 5.960e+01,
       6.061e+01, 6.162e+01, 6.263e+01, 6.364e+01, 6.465e+01, 6.566e+01,
       6.667e+01, 6.768e+01, 6.869e+01, 6.970e+01, 7.071e+01, 7.172e+01,
       7.273e+01, 7.374e+01, 7.475e+01, 7.576e+01, 7.677e+01, 7.778e+01,
       7.879e+01, 7.980e+01, 8.081e+01, 8.182e+01, 

Чтобы отобрать среди данного массива гиперпараметров лучший, воспользуйтесь конструкцией `GridSearchCV` из `sklearn`

In [115]:
param_grid = {
    'LR__C': alphas
}

### Your code is here

from sklearn.model_selection import GridSearchCV

gs = GridSearchCV(pipe, param_grid=param_grid, scoring='accuracy')
gs.fit(X_2_train, y_2_train)

GridSearchCV(estimator=Pipeline(steps=[('scaler', StandardScaler()),
                                       ('LR', LogisticRegression())]),
             param_grid={'LR__C': array([1.000e-02, 1.020e+00, 2.030e+00, 3.040e+00, 4.050e+00, 5.060e+00,
       6.070e+00, 7.080e+00, 8.090e+00, 9.100e+00, 1.011e+01, 1.112e+01,
       1.213e+01, 1.314e+01, 1.415e+01, 1.516e+01, 1.617e+01, 1.718e+01,
       1.819e+01, 1.920e+01, 2.021e+01, 2.122e+01, 2.223e...
       6.667e+01, 6.768e+01, 6.869e+01, 6.970e+01, 7.071e+01, 7.172e+01,
       7.273e+01, 7.374e+01, 7.475e+01, 7.576e+01, 7.677e+01, 7.778e+01,
       7.879e+01, 7.980e+01, 8.081e+01, 8.182e+01, 8.283e+01, 8.384e+01,
       8.485e+01, 8.586e+01, 8.687e+01, 8.788e+01, 8.889e+01, 8.990e+01,
       9.091e+01, 9.192e+01, 9.293e+01, 9.394e+01, 9.495e+01, 9.596e+01,
       9.697e+01, 9.798e+01, 9.899e+01, 1.000e+02])},
             scoring='accuracy')

In [127]:
from sklearn.metrics import classification_report

print(classification_report(y_true=y_2_test, y_pred=gs.predict(X_2_test), digits=3))

              precision    recall  f1-score   support

           0      0.640     0.633     0.637     25025
           1      0.637     0.644     0.640     24975

    accuracy                          0.638     50000
   macro avg      0.638     0.638     0.638     50000
weighted avg      0.638     0.638     0.638     50000



In [130]:
gs.best_params_

{'LR__C': 13.14}

In [124]:
from sklearn.metrics import classification_report

print(classification_report(y_true=y_2_test, y_pred=gs.predict(X_2_test), digits=3))

              precision    recall  f1-score   support

           0      0.640     0.633     0.637     25025
           1      0.637     0.644     0.640     24975

    accuracy                          0.638     50000
   macro avg      0.638     0.638     0.638     50000
weighted avg      0.638     0.638     0.638     50000



#### Задание 4: Бонус

Как вы знаете, подбор признаков является одной из самых важных частей машинного обучения. Сейчас вы попробуете на основе имеющихся признаков сгенерировать новые. В качестве новых признаков будем использовать мономы 2 степени. Можно использовать регуляризацию различного рода, выбор энкодера тоже за вами. Ваша задача - добиться качества `0.65` на тестовой выборке

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
data = pd.read_csv('Data/train.csv')

C:\Users\merku\AppData\Local\Temp\ipykernel_27612\3943152542.py:1: DtypeWarning: Columns (28) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv('Data/train.csv')


In [3]:
data.head()

,MachineIdentifier,ProductName,EngineVersion,AppVersion,AvSigVersion,IsBeta,RtpStateBitfield,IsSxsPassiveMode,DefaultBrowsersIdentifier,AVProductStatesIdentifier,...,Census_FirmwareVersionIdentifier,Census_IsSecureBootEnabled,Census_IsWIMBootEnabled,Census_IsVirtualDevice,Census_IsTouchEnabled,Census_IsPenCapable,Census_IsAlwaysOnAlwaysConnectedCapable,Wdft_IsGamer,Wdft_RegionIdentifier,HasDetections
0,0000028988387b115f69f31a3bf04f09,win8defender,1.1.15100.1,4.18.1807.18075,1.273.1735.0,0,7.0,0,NaN,53447.0,...,36144.0,0,NaN,0.0,0,0,0.0,0.0,10.0,0
1,000007535c3f730efa9ea0b7ef1bd645,win8defender,1.1.14600.4,4.13.17134.1,1.263.48.0,0,7.0,0,NaN,53447.0,...,57858.0,0,NaN,0.0,0,0,0.0,0.0,8.0,0
2,000007905a28d863f6d0d597892cd692,win8defender,1.1.15100.1,4.18.1807.18075,1.273.1341.0,0,7.0,0,NaN,53447.0,...,52682.0,0,NaN,0.0,0,0,0.0,0.0,3.0,0
3,00000b11598a75ea8ba1beea8459149f,win8defender,1.1.15100.1,4.18.1807.18075,1.273.1527.0,0,7.0,0,NaN,53447.0,...,20050.0,0,NaN,0.0,0,0,0.0,0.0,3.0,1
4,000014a5f00daa18e76b81417eeb99fc,win8defender,1.1.15100.1,4.18.1807.18075,1.273.1379.0,0,7.0,0,NaN,53447.0,...,19844.0,0,0.0,0.0,0,0,0.0,0.0,1.0,1


In [4]:
data.isnull().sum()

MachineIdentifier                             0
ProductName                                   0
EngineVersion                                 0
AppVersion                                    0
AvSigVersion                                  0
                                           ... 
Census_IsPenCapable                           0
Census_IsAlwaysOnAlwaysConnectedCapable    1621
Wdft_IsGamer                               6737
Wdft_RegionIdentifier                      6737
HasDetections                                 0
Length: 83, dtype: int64

In [18]:
def get_redundant_pairs(df):
    pairs_to_drop = set()
    cols = df.columns
    for i in range(0, df.shape[1]):
        for j in range(0, i + 1):
            pairs_to_drop.add((cols[i], cols[j]))
    return pairs_to_drop


def get_top_abs_correlations(df, n=5):
    au_corr = df.corr().abs().unstack()
    labels_to_drop = get_redundant_pairs(df)
    au_corr = au_corr.drop(labels=labels_to_drop).sort_values(ascending=False)
    return au_corr[0:n]


print("Top Absolute Correlations")
print(get_top_abs_correlations(data[data.select_dtypes('number').columns.to_list()], 7))

Top Absolute Correlations
MachineIdentifier             HasDetections                              1.000000
Census_OSEdition              Census_OSSkuName                           0.996821
OsSuite                       SkuEdition                                 0.915596
Census_PowerPlatformRoleName  Census_IsAlwaysOnAlwaysConnectedCapable    0.862911
OsPlatformSubRelease          Census_OSBranch                            0.792457
Census_MDC2FormFactor         Census_PowerPlatformRoleName               0.789716
                              Census_IsAlwaysOnAlwaysConnectedCapable    0.734013
dtype: float64


In [19]:
def correlation(dataset: pd.DataFrame, threshold: float, columns: list | None):
    col_corr = set()
    corr_matrix = dataset[columns].corr() if columns else dataset.corr()

    for i in range(len(corr_matrix.columns)):
        for j in range(i):
            if (abs(corr_matrix.iloc[i, j]) >= threshold) and (corr_matrix.columns[j] not in col_corr):
                colname = corr_matrix.columns[i]
                col_corr.add(colname)
                if colname in dataset.columns:
                    del dataset[colname]


correlation(dataset=data, threshold=0.75, columns=data.select_dtypes('number').columns.to_list())

print("Top Absolute Correlations")
print(get_top_abs_correlations(data[data.select_dtypes('number').columns.to_list()], 7))

Top Absolute Correlations
Census_MDC2FormFactor      Census_IsAlwaysOnAlwaysConnectedCapable    0.734013
AVProductStatesIdentifier  AVProductsInstalled                        0.633168
Census_ProcessorCoreCount  Census_TotalPhysicalRAM                    0.609418
CountryIdentifier          GeoNameIdentifier                          0.595886
EngineVersion              AvSigVersion                               0.594502
OsBuild                    OsPlatformSubRelease                       0.582566
OsPlatformSubRelease       Census_OSVersion                           0.536103
dtype: float64


In [7]:
num_col = np.array(data.drop('HasDetections', axis=1).select_dtypes('number').columns.to_list())

from sklearn.feature_selection import VarianceThreshold

vt = VarianceThreshold(threshold=0)
res = vt.fit_transform(X=data[num_col])

data.drop(np.setdiff1d(num_col, vt.get_feature_names_out()), inplace=True, axis=1)

In [17]:
# num_col = data.drop('HasDetections', axis=1).select_dtypes('number').columns.to_list()
# 
# for col in num_col:
#     fig = plt.figure()
#     sns.boxplot(y=col, x=data['HasDetections'].astype('category'), data=data)
#     plt.show()

In [9]:
categorical_columns = data.select_dtypes('object').columns.to_list()

data[[*categorical_columns, 'HasDetections']].groupby('HasDetections').count().T

HasDetections,0,1
MachineIdentifier,99940,100060
ProductName,99940,100060
EngineVersion,99940,100060
AppVersion,99940,100060
AvSigVersion,99940,100060
Platform,99940,100060
Processor,99940,100060
OsVer,99940,100060
OsPlatformSubRelease,99940,100060
OsBuildLab,99939,100060


In [10]:
data.dropna(thresh=data.shape[0] / 2, axis=1, inplace=True)

In [11]:
null_categorical_columns = data.select_dtypes('object').loc[:,
                           data.select_dtypes('object').isna().sum() != 0].columns.to_list()

data[null_categorical_columns] = data[null_categorical_columns].fillna('NaN')

In [12]:
null_numeric_columns = data.select_dtypes('number').loc[:,
                       data.select_dtypes('number').isna().sum() != 0].columns.to_list()

grouped_mean = data.groupby('HasDetections')[null_numeric_columns].transform("mean")
data[null_numeric_columns] = data[null_numeric_columns].fillna(grouped_mean)

In [13]:
data.drop_duplicates(inplace=True)

In [14]:
less_than_5_cols = data.select_dtypes('object').loc[:, data.nunique() <= 5].columns.values
more_than_5_cols = data.select_dtypes('object').loc[:, data.nunique() > 5].columns.values

data = pd.get_dummies(data, columns=less_than_5_cols, drop_first=True)

for col in more_than_5_cols:
    mean_target = data.groupby(col)['HasDetections'].mean()
    data[col] = data[col].map(mean_target)

In [22]:
from sklearn.model_selection import train_test_split

X = data.drop(columns=['HasDetections'])
y = data['HasDetections']

X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                    test_size=0.3,
                                                    shuffle=True,
                                                    random_state=1)

KeyError: "['HasDetections'] not found in axis"

In [21]:
from sklearn.model_selection import cross_validate
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import PolynomialFeatures

new_pipe = Pipeline([('scaler', StandardScaler()),
                     ('poly', PolynomialFeatures(degree=2)),
                     ('LR', LogisticRegression(penalty='l2', C=13.14))])

cv_result_pipe = cross_validate(new_pipe, X, y, scoring='accuracy',
                                cv=4, return_train_score=True)

print(f"Accuracy на трейне: {np.mean(cv_result_pipe['train_score']).round(3)}")
print(f"Accuracy на тесте: {np.mean(cv_result_pipe['test_score']).round(3)}")



KeyboardInterrupt

